# AI Toolkit by Ostris
## FLUX.1-dev Training


In [1]:
!nvidia-smi

Tue Jan  7 16:30:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             42W /  400W |       1MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!mkdir -p ~/ai-toolkit/dataset

Put your image dataset in the `~/ai-toolkit/dataset` folder

In [3]:
!cd ~/ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

Submodule 'repositories/batch_annotator' (https://github.com/ostris/batch-annotator) registered for path 'repositories/batch_annotator'
Submodule 'repositories/ipadapter' (https://github.com/tencent-ailab/IP-Adapter.git) registered for path 'repositories/ipadapter'
Submodule 'repositories/leco' (https://github.com/p1atdev/LECO) registered for path 'repositories/leco'
Submodule 'repositories/sd-scripts' (https://github.com/kohya-ss/sd-scripts.git) registered for path 'repositories/sd-scripts'
Cloning into '/home/bhara/ai-toolkit/repositories/batch_annotator'...
Cloning into '/home/bhara/ai-toolkit/repositories/ipadapter'...
Cloning into '/home/bhara/ai-toolkit/repositories/leco'...
Cloning into '/home/bhara/ai-toolkit/repositories/sd-scripts'...
Submodule path 'repositories/batch_annotator': checked out '420e142f6ad3cc14b3ea0500affc2c6c7e7544bf'
Submodule 'repositories/controlnet' (https://github.com/lllyasviel/ControlNet-v1-1-nightly.git) registered for path 'repositories/batch_annotat

## Model License
Training currently only works with FLUX.1-dev. Which means anything you train will inherit the non-commercial license. It is also a gated model, so you need to accept the license on HF before using it. Otherwise, this will fail. Here are the required steps to setup a license.

Sign into HF and accept the model access here [black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)

[Get a READ key from huggingface](https://huggingface.co/settings/tokens/new?) and place it in the next cell after running it.

In [4]:
import getpass
import os

# Prompt for the token
hf_token = ""

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

HF_TOKEN environment variable has been set.


In [7]:
import os
import sys
sys.path.append('/home/bhara/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Setup

This is your config. It is documented pretty well. Normally you would do this as a yaml file, but for colab, this will work. This will run as is without modification, but feel free to edit as you want.

In [ ]:
from collections import OrderedDict

# Define your parameters
parameters = {
    "STEPS": 5000,
    "LEARNING_RATE": 1e-4,
    "DATASET": "og-chair-bg",
    "TRAINING_TYPE": "balanced",
    "LORA_RANK": 16,
    "SAVE_EVERY": 250,
    "CAPTION_DROPOUT_RATE": 0.05,
    "SAMPLE_EVERY": 250,
    "SAMPLING_PROMPT": [
        'woman with red hair, playing chess at the park, bomb going off in the background',
        'a woman holding a coffee cup, in a beanie, sitting at a cafe',
        'a horse is a DJ at a night club, fish eye lens, smoke machine, lazer lights, holding a martini',
        'a man showing off his cool new t shirt at the beach, a shark is jumping out of the water in the background',
        'a bear building a log cabin in the snow covered mountains',
        'woman playing the guitar, on stage, singing a song, laser lights, punk rocker',
        'hipster man with a beard, building a chair, in a wood shop',
        'photo of a man, white background, medium shot, modeling clothing, studio lighting, white backdrop',
        'a man holding a sign that says, "this is a sign"',
        'a bulldog, in a post apocalyptic world, with a shotgun, in a leather jacket, in a desert, with a motorcycle'
    ],
    "SEED": 42,
    "SAMPLING_STEPS": 20,
    "LAYERS": [
        "transformer.single_transformer_blocks.7.proj_out",
        "transformer.single_transformer_blocks.12.proj_out",
        "transformer.single_transformer_blocks.16.proj_out",
        "transformer.single_transformer_blocks.20.proj_out"
    ]
}

try:
    layers = ""
    for i in parameters["LAYERS"]:
        layers += f"-{i.split('.')[-2]}"
    print(layers)
    lora_name = f"{parameters['DATASET']}-steps{parameters['STEPS']}-lr{parameters['LEARNING_RATE']}-lrank{parameters['LORA_RANK']}-b1_layers{layers}"
except:
    lora_name = f"{parameters['DATASET']}-steps{parameters['STEPS']}-lr{parameters['LEARNING_RATE']}-lrank{parameters['LORA_RANK']}-b1_full-layer"
    
print(lora_name)

In [ ]:
# Generate the configuration dynamically
job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
        ('name', lora_name),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                ('training_folder', '~/ai-toolkit/output'),
                ('performance_log_every', 1000),
                ('device', 'cuda:0'),
                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', parameters["LORA_RANK"]),
                    ('linear_alpha', 16)
                ])),
                ('save', OrderedDict([
                    ('dtype', 'float16'),
                    ('save_every', parameters["SAVE_EVERY"]),
                    ('max_step_saves_to_keep', 4)
                ])),
                ('datasets', [
                    OrderedDict([
                        ('folder_path', f'/home/bhara/ai-toolkit/dataset/{parameters["DATASET"]}'),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', parameters["CAPTION_DROPOUT_RATE"]),
                        ('shuffle_tokens', False),
                        ('cache_latents_to_disk', True),
                        ('resolution', [512, 768, 1024])
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', 1),
                    ('steps', parameters["STEPS"]),
                    ('gradient_accumulation_steps', 1),
                    ('train_unet', True),
                    ('train_text_encoder', False),
                    ('content_or_style', parameters["TRAINING_TYPE"]),
                    ('gradient_checkpointing', True),
                    ('noise_scheduler', 'flowmatch'),
                    ('optimizer', 'adamw8bit'),
                    ('lr', parameters["LEARNING_RATE"]),
                    ('skip_first_sample', True),
                    ('ema_config', OrderedDict([
                        ('use_ema', True),
                        ('ema_decay', 0.99)
                    ])),
                    ('dtype', 'bf16')
                ])),
                ('model', OrderedDict([
                    ('name_or_path', 'black-forest-labs/FLUX.1-dev'),
                    ('is_flux', True),
                    ('quantize', False),
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),
                    ('sample_every', parameters["SAMPLE_EVERY"]),
                    ('width', 1024),
                    ('height', 1024),
                    ('prompts', parameters["SAMPLING_PROMPT"]),
                    ('neg', ''),
                    ('seed', parameters["SEED"]),
                    ('walk_seed', True),
                    ('guidance_scale', 4),
                    ('sample_steps', parameters["SAMPLING_STEPS"])
                ]))
            ])
        ])
    ])),
    ('meta', OrderedDict([
        ('name', lora_name),
        ('version', '1.0')
    ]))
])

# Print or use the updated configuration
print(job_to_run)

## Run it

Below does all the magic. Check your folders to the left. Items will be in output/LoRA/your_name_v1 In the samples folder, there are preiodic sampled. This doesnt work great with colab. They will be in /content/output

In [ ]:
run_job(job_to_run)


## Done

Check your ourput dir and get your slider
